In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
df = pd.read_excel('/home/sam/tirocinio/DB PAROTIDE DEF_corretto.xlsx', sheet_name='Sheet7')
df = df.loc[df['ESCLUDERE'] == 0]
df.reset_index(inplace = True)
df = df.loc[df['COD ISTO'] != 6]
df.reset_index(inplace = True)
df = df[['Segni macro malignità', 'ADC', 'TIC type', 'T2','COD ISTO']]
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df.head(10)

,Segni macro malignità,ADC,TIC type,T2,COD ISTO
0,0,1.60,A,1,3
1,0,1.00,B,0,4
2,0,1.20,C,0,3
3,0,0.77,B,1,4
4,0,0.80,C,1,1
5,1,0.70,C,1,1
6,0,0.90,B,1,4
7,0,0.70,B,1,4
8,0,0.80,B,1,4
9,0,1.40,A,0,3


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Segni macro malignità  103 non-null    int64  
 1   ADC                    103 non-null    float64
 2   TIC type               103 non-null    object 
 3   T2                     103 non-null    int64  
 4   COD ISTO               103 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 4.1+ KB


In [4]:
df['COD ISTO'].value_counts()

3    53
4    27
1    12
5     6
2     5
Name: COD ISTO, dtype: int64

In [5]:
#kruskall wallis su adc-->vedi quello che hai scritto nella parte vecchia
gruppo_1 = np.array([value for i, value in enumerate(df['ADC']) if df['COD ISTO'].iloc[i] == 1])
gruppo_2 = np.array([value for i, value in enumerate(df['ADC']) if df['COD ISTO'].iloc[i] == 2])
gruppo_3 = np.array([value for i, value in enumerate(df['ADC']) if df['COD ISTO'].iloc[i] == 3])
gruppo_4 = np.array([value for i, value in enumerate(df['ADC']) if df['COD ISTO'].iloc[i] == 4])
gruppo_5 = np.array([value for i, value in enumerate(df['ADC']) if df['COD ISTO'].iloc[i] == 5])
stats.kruskal(gruppo_1, gruppo_2, gruppo_3, gruppo_4, gruppo_5)

KruskalResult(statistic=72.34297963883975, pvalue=7.263259102017699e-15)

In [6]:
#senza gruppo 5
stats.kruskal(gruppo_1, gruppo_2, gruppo_3, gruppo_4)

KruskalResult(statistic=69.49221476991501, pvalue=5.482584679436209e-15)

In [7]:
#solo tra 1 e 4
stats.kruskal(gruppo_1, gruppo_4)

KruskalResult(statistic=2.430432557956478, pvalue=0.11900005739824361)

In [8]:
#chi square su tic type

In [9]:
#contingency table for chi square test on TIC type

#questa tabella non verrà impiegata per il test, è solo per visualizzazione
contingency_table_TIC = pd.crosstab(index = df['COD ISTO'],
                                     columns = df['TIC type'],
                                     colnames = [''],
                                     margins = True)

#l'effettiva tabella impiegata nel test (cambia solo che non sono presenti i valori marginali)
cont_TIC = pd.crosstab(index = df['COD ISTO'],
                         columns = df['TIC type'],
                         colnames = [''])

contingency_table_TIC = contingency_table_TIC.reindex([1,2,3,4,5, 'All'])
cont_TIC = cont_TIC.reindex([1,2,3,4,5])
contingency_table_TIC

,A,B,C,All
COD ISTO,,,,
1,0,0,12,12
2,0,3,2,5
3,41,0,12,53
4,0,27,0,27
5,0,0,6,6
All,41,30,32,103


In [10]:
#chi square per TIC type tra tutti e 5 i gruppi

stat, p, dof, expected = stats.chi2_contingency(cont_TIC)

print('Expected frequencies')
print(pd.DataFrame(expected,
                  index = cont_TIC.index,
                  columns = cont_TIC.columns).apply(lambda n: np.round(n,3)))
print()
print('Degrees of freedom: {}\nChi stat: {}\np-value: {}\n'.format(dof,stat,p))

Expected frequencies
               A       B       C
COD ISTO                        
1          4.777   3.495   3.728
2          1.990   1.456   1.553
3         21.097  15.437  16.466
4         10.748   7.864   8.388
5          2.388   1.748   1.864

Degrees of freedom: 8
Chi stat: 144.8170283018868
p-value: 2.358923272431205e-27



In [11]:
#fisher excact


In [12]:
import FisherExact
help(FisherExact)

Help on package FisherExact:

NAME
    FisherExact

DESCRIPTION
    # FisherExact Copyright (C) 2016  Emmanuel Noutahi
    # This program is free software: you can redistribute it and/or modify
    # it under the terms of the GNU General Public License as published by
    # the Free Software Foundation, either version 3 of the License, or
    # (at your option) any later version.

PACKAGE CONTENTS
    Fisher
    statlib (package)

FUNCTIONS
    fisher_exact(table, alternative='two-sided', hybrid=False, midP=False, simulate_pval=False, replicate=2000, workspace=300, attempt=3, seed=None)
        Performs a Fisher exact test on a mxn contingency table.
        Parameters
        ----------
        table : array_like of ints
            A mxn contingency table.  Elements should be non-negative integers.
        alternative : {'two-sided', 'less', 'greater'}, optional
            Which alternative hypothesis to the null hypothesis the test uses.
            Default is 'two-sided'.  Only us

In [13]:
#eseguo il fisher exact per TIC_type su tutti e 5 i gruppi

p_value = FisherExact.fisher_exact(cont_TIC)
print('fisher exact test p-value: {}'.format(p_value))

fisher exact test p-value: 0.0


In [14]:
#chi square e fisher excat sul t2

In [15]:
contingency_table_T2 = pd.crosstab(index = df['COD ISTO'],
                                     columns = df['T2'],
                                     colnames = [''],
                                     margins = True)

#l'effettiva tabella impiegata nel test (cambia solo che non sono presenti i valori marginali)
cont_T2 = pd.crosstab(index = df['COD ISTO'],
                         columns = df['T2'],
                         colnames = [''])

contingency_table_T2 = contingency_table_T2.reindex([1,2,3,4,5, 'All'])
cont_T2 = cont_T2.reindex([1,2,3,4,5])
contingency_table_T2

,0,1,All
COD ISTO,,,
1,1,11,12
2,2,3,5
3,37,16,53
4,4,23,27
5,5,1,6
All,49,54,103


In [16]:
stat, p, dof, expected = stats.chi2_contingency(cont_T2)

print('Expected frequencies')
print(pd.DataFrame(expected,
                  index = cont_T2.index,
                  columns = cont_T2.columns).apply(lambda n: np.round(n,3)))
print()
print('Degrees of freedom: {}\nChi stat: {}\np-value: {}\n'.format(dof,stat,p))

Expected frequencies
               0       1
COD ISTO                
1          5.709   6.291
2          2.379   2.621
3         25.214  27.786
4         12.845  14.155
5          2.854   3.146

Degrees of freedom: 4
Chi stat: 32.725524003374154
p-value: 1.3594447636085382e-06



In [17]:
#dovrei usare fisher exact
p_value = FisherExact.fisher_exact(cont_T2)
print('fisher exact test p-value: {}'.format(p_value))

fisher exact test p-value: 0.0


In [18]:
#chi square tra 4 e 1

df_alt = df.loc[(df['COD ISTO'] == 1) | (df['COD ISTO'] == 4)]

contingency_table_T2_1_4 = pd.crosstab(index = df_alt['COD ISTO'],
                                     columns = df_alt['T2'],
                                     colnames = [''],
                                     margins = True)

#l'effettiva tabella impiegata nel test (cambia solo che non sono presenti i valori marginali)
cont_T2_1_4 = pd.crosstab(index = df_alt['COD ISTO'],
                                     columns = df_alt['T2'],
                                     colnames = [''])

contingency_table_T2_1_4 = contingency_table_T2_1_4.reindex([1,4,'All'])
cont_T2_1_4 = cont_T2_1_4.reindex([1,4])
contingency_table_T2_1_4

,0,1,All
COD ISTO,,,
1,1,11,12
4,4,23,27
All,5,34,39


In [19]:
stat, p, dof, expected = stats.chi2_contingency(cont_T2_1_4)

print('Expected frequencies')
print(pd.DataFrame(expected,
                  index = cont_T2_1_4.index,
                  columns = cont_T2_1_4.columns).apply(lambda n: np.round(n,3)))
print()
print('Degrees of freedom: {}\nChi stat: {}\np-value: {}\n'.format(dof,stat,p))

Expected frequencies
              0       1
COD ISTO               
1         1.538  10.462
4         3.462  23.538

Degrees of freedom: 1
Chi stat: 0.0015931372549019664
p-value: 0.9681615911503386



In [20]:
#uso fisher e.

#applico fisher exact implementato in scipy
oddr, p_value = stats.fisher_exact(cont_T2_1_4)
print('fisher exact test p-value: {}'.format(p_value))


fisher exact test p-value: 1.0


In [21]:
# applico fisher exact dal paccheto "non ufficiale"
p_value = FisherExact.fisher_exact(cont_T2_1_4)
print('fisher exact test p-value: {}'.format(p_value))

fisher exact test p-value: 1.0


In [22]:
#chi square tra 3 e 5

df_alt = df.loc[(df['COD ISTO'] == 5) | (df['COD ISTO'] == 3)]

contingency_table_T2_3_5 = pd.crosstab(index = df_alt['COD ISTO'],
                                     columns = df_alt['T2'],
                                     colnames = [''],
                                     margins = True)

#l'effettiva tabella impiegata nel test (cambia solo che non sono presenti i valori marginali)
cont_T2_3_5 = pd.crosstab(index = df_alt['COD ISTO'],
                                     columns = df_alt['T2'],
                                     colnames = [''])

contingency_table_T2_3_5 = contingency_table_T2_3_5.reindex([3,5,'All'])
cont_T2_3_5 = cont_T2_3_5.reindex([3,5])
contingency_table_T2_3_5

,0,1,All
COD ISTO,,,
3,37,16,53
5,5,1,6
All,42,17,59


In [23]:
stat, p, dof, expected = stats.chi2_contingency(cont_T2_3_5)

print('Expected frequencies')
print(pd.DataFrame(expected,
                  index = cont_T2_3_5.index,
                  columns = cont_T2_3_5.columns).apply(lambda n: np.round(n,3)))
print()
print('Degrees of freedom: {}\nChi stat: {}\np-value: {}\n'.format(dof,stat,p))

Expected frequencies
               0       1
COD ISTO                
3         37.729  15.271
5          4.271   1.729

Degrees of freedom: 1
Chi stat: 0.04735809418106868
p-value: 0.8277258629991427



In [24]:
#applico fisher exact implementato in scipy
oddr, p_value = stats.fisher_exact(cont_T2_3_5)
print('fisher exact test p-value: {}'.format(p_value))

fisher exact test p-value: 0.6621541633692133


In [25]:
#applico fisher exact non ufficiale

p_value = FisherExact.fisher_exact(cont_T2_3_5)
print('fisher exact test p-value: {}'.format(p_value))

fisher exact test p-value: 0.6621541633692133


In [26]:
#i risultati li vedi domani :)